## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import random

import os

current_wd = str(os.getcwd())
parent_parent_parent = os.path.dirname(os.path.dirname(os.path.dirname(current_wd)))
os.chdir(parent_parent_parent)

from MyImports import *
hv.extension('plotly')

os.chdir(current_wd)

seed = 42
pyro.set_rng_seed(seed)
torch.manual_seed(seed);

# Objective - continuous

In [ ]:
def _generator():
    res = torch.tensor(0.)
    while(True):
        yield(torch.sin(res)*2.5)
        res += 0.3
gen = _generator()

def get_current_context():
    return next(gen)

In [ ]:
hv.extension('bokeh', logo=False)

x1 = torch.linspace(0,5,100, dtype=torch.float64)
x2 = torch.linspace(-2.5,2.5,100, dtype=torch.float64)
xx1, xx2 = [a.flatten() for a in torch.meshgrid((x1, x2), indexing='xy')]

y_obj = Objective2D(x1=xx1, x2=xx2, noise=0.0).data

obj_plot = hv.HeatMap([(x.item(), y.item(), z.item()) for x,y,z in zip(xx1, xx2, y_obj)],
                      label='Objective Curve',
                     ).opts(cmap='RdBu_r',
                            height=400,
                            width=400,
                            colorbar=True,
                            title='2D Objective Function',
                            xlabel='continuous variable',
                            ylabel='continuous context',
                            ylim=(-2.45,2.45))
context = get_current_context()
context_val = hv.Curve((x1, context.repeat(len(x1),)), label='Context value').opts(color='green')
(obj_plot * context_val).opts(height=380,show_legend=True, legend_position='bottom')

In [ ]:
def getHmapPlots(init_model,
                 X_test,
                 objective,
                 num_iter,
                 show_legend=True,
                 xlabel=None, ylabel=None,
                 init_samples=None):
    
    models = {0: init_model}
    model_dict = {0: simplePlot(*extractPlotData(X_test.view(-1,1), init_model),
                                show_legend=show_legend,
                                xlabel=xlabel,
                                ylabel=ylabel,
                                init_samples=init_samples)}
    print('Iteration 0: \n noise: {:.2f}, l: {:.2f}, lambda: {:.2f}'.format(init_model.noise,
                                                                             init_model.kernel.lengthscale_unconstrained.exp(),
                                                                             init_model.kernel.variance_unconstrained.exp()))
    
    lengthscale.append(init_model.kernel.lengthscale_unconstrained.exp())
    outputscale.append(init_model.kernel.variance_unconstrained.exp())
    noise_.append(init_model.noise)
    
    model_dict[0] = hv.Layout(model_dict[0][0] + model_dict[0][1]).cols(1)
    
    for i in range(1,num_iter+1):
        previous = models[i-1]
        next_sample = previous.next_sample(X_test.view(-1,1))
        mod = PyroBO(X=previous.X,
                     y=previous.y,
                     kernel=previous.kernel,
                     noise=previous.noise,
                     jitter=previous.jitter,
                     acq_fun=previous.acq_fun)
        
        context = get_current_context()
        print('Context value: ', context)
        
        mod.update()
        mod.add_Observation(next_sample, objective(x1=next_sample, x2=context, noise=noise))
        mod.update()
        models[i] = mod
        model_dict[i] = simplePlot(*extractPlotData(X_test.view(-1,1), mod),
                                   show_legend=show_legend,
                                   xlabel=xlabel,
                                   ylabel=ylabel,
                                   init_samples=init_samples)
        model_dict[i] = hv.Layout(model_dict[i][0].opts(height=300)
                                  + model_dict[i][1]).cols(1)
        
        print('Iteration {}: \n noise: {:.2f}, l: {:.2f}, lambda: {:.2f}'.format(i,
                                                                                  init_model.noise,
                                                                                  init_model.kernel.lengthscale_unconstrained.exp(),
                                                                                  init_model.kernel.variance_unconstrained.exp()))
        
        lengthscale.append(init_model.kernel.lengthscale_unconstrained.exp())
        outputscale.append(init_model.kernel.variance_unconstrained.exp())
        noise_.append(init_model.noise)
        
    return model_dict

## Classical BO

In [ ]:
pyro.set_rng_seed(55)

noise = 0.0
init_num = 20
input_dim = 1
num_iter = 30
kappa = 10.

X_init = torch.cat((torch.rand(init_num,1, dtype=torch.float64)*5,
                    torch.rand(init_num,1, dtype=torch.float64)*5-2.5),
                   dim=1) #torch.tensor([[0.49, -1.5], [3.4, 2.1], [4.52, 0.3]], dtype=torch.float64)

y_init = Objective2D(x1=X_init.T[0].T, x2=X_init.T[1].T, noise=noise)

cl_model = PyroBO(X_init.T[0:1].T, y_init, kernel=gp.kernels.RBF(input_dim=input_dim),
                  acq_fun=ConfidenceBound(input_dim=input_dim,
                                          kappa=kappa,
                                          maximize=True))
cl_model.update()

In [ ]:
lengthscale = []
outputscale = []
noise_ = []

X_test = torch.linspace(0,5,100, dtype=torch.float64).reshape(-1,1)
hmap1 = hv.HoloMap(getHmapPlots(cl_model, X_test, Objective2D, num_iter, xlabel='continuous variable', ylabel=' ',
                                init_samples=(X_init.T[0], y_init)),
                   kdims='iteration')

hv.output(hmap1.collate(), widget_location='top')

In [ ]:
hv.extension('plotly')

lengthscale_cl_plot = hv.Curve(((torch.tensor(lengthscale))),
                                label='classical BO').opts(xlabel='iteration', ylabel=' ', height=200)
outputscale_cl_plot = hv.Curve((torch.tensor(outputscale)),
                                label='classical BO').opts(xlabel='iteration', ylabel=' ', height=200)
noise_cl_plot = hv.Curve((torch.tensor(noise_)),
                          label='classical BO').opts(xlabel='iteration', ylabel=' ', height=200)

In [ ]:
hv.save(hmap1.collate(), 'contextClassic.auto')

## Classical BO - noisy

In [ ]:
noise = 0.5

y_init = Objective2D(x1=X_init.T[0].T, x2=X_init.T[1].T, noise=noise)

cln_model = PyroBO(X_init.T[0:1].T, y_init, kernel=gp.kernels.RBF(input_dim=input_dim),
                  acq_fun=ConfidenceBound(input_dim=input_dim,
                                          kappa=kappa,
                                          maximize=True))
cln_model.update()

lengthscale = []
outputscale = []
noise_ = []

X_test = torch.linspace(0,5,100, dtype=torch.float64).reshape(-1,1)
hmap2 = hv.HoloMap(getHmapPlots(cln_model, X_test, Objective2D, num_iter, xlabel='continuous variable', ylabel=' ',
                                init_samples=(X_init.T[0], y_init)),
                   kdims='iteration')

hv.output(hmap2.collate(), widget_location='top')

In [ ]:
hv.extension('plotly')

lengthscale_cln_plot = hv.Curve(((torch.tensor(lengthscale))),
                                label='classical BO').opts(xlabel='iteration', ylabel=' ', height=200)
outputscale_cln_plot = hv.Curve((torch.tensor(outputscale)),
                                label='classical BO').opts(xlabel='iteration', ylabel=' ', height=200)
noise_cln_plot = hv.Curve((torch.tensor(noise_)),
                          label='classical BO').opts(xlabel='iteration', ylabel=' ', height=200)

In [ ]:
hv.save(hmap2.collate(), 'contextClassic_noisy.auto')

## Extended BO

In [ ]:
hv.extension('bokeh')

In [ ]:
noise = 0.
input_dim = 2

y_init = Objective2D(x1=X_init.T[0].T, x2=X_init.T[1].T, noise=noise)
kernel = gp.kernels.RBF(input_dim=input_dim)

bo_model = PyroBO(X_init, y_init, kernel=kernel, acq_fun=ConfidenceBound(input_dim=input_dim,
                                                                         kappa=kappa,
                                                                         maximize=True))
bo_model.update()

In [ ]:
X_test = torch.cat((xx1.reshape(-1,1), xx2.reshape(-1,1)), dim=1)

In [ ]:
acqs = OrderedDict()

In [ ]:
def getHeatMap(init_model, X_test, update_context, objective, num_iter=2, precision=100, init_samples=None,
               fontsize = {'title': 15, 'xlabel': 14, 'ylabel': 14, 'legend': 14}):
    model_dict = OrderedDict()
    
    for i in range(num_iter+1):
        
        if i != 0:
            init_model.add_Observation(x_next, objective(x1=x_next.T[0:1], x2=x_next.T[1:], noise=noise))
            init_model.update()
            
        print('Iteration {}:\n noise: {:.2f}, l: {:.2f}, lambda: {:.2f}'.format(i,
                                                                                 init_model.noise,
                                                                                 init_model.kernel.lengthscale_unconstrained.exp(),
                                                                                 init_model.kernel.variance_unconstrained.exp()))

        lengthscale.append(init_model.kernel.lengthscale_unconstrained.exp())
        outputscale.append(init_model.kernel.variance_unconstrained.exp())
        noise_.append(init_model.noise)

        context = update_context()
        print('Context value: ', context)
        x_test_context = torch.cat((X_test.T[0].T[:precision].reshape(-1,1), context.repeat(len(x1), 1)), dim=1)
        context_marker = hv.Curve((x_test_context.T[0],
                                   x_test_context.T[1]),
                                  label='Current context').opts(color='green')

        acqs[i] = (init_model.get_acq_values(x_test_context).data, context)
        x_next = init_model.next_sample(x_test_context)

        next_point = hv.Points(x_next, label='Next sample').opts(color='red', size=6)

        mean_pred = init_model(X_test)[0].data
        data = [(x.item(), y.item(), z.item()) for x,y,z in zip(X_test.T[0], X_test.T[1], mean_pred)]

        train_points = hv.Points((init_model.X.T[0],
                                  init_model.X.T[1]),
                                 label='Iteration Points').opts(color='black', size=5)

        if init_samples is not None:
            train_points = train_points * hv.Points((init_samples[0].T[0],
                                                     init_samples[0].T[1]),
                                                    label='Initial Points').opts(color='orange', size=5)

            mean_plot = hv.HeatMap(data).opts(cmap='RdBu_r',
                                              zlim=(y_obj.min(), y_obj.max()),
                                              width=580,
                                              colorbar=True,
                                              title='Mean prediction',
                                              xlabel='continous variable',
                                              ylabel='continous context') * train_points * context_marker

            acq_values = init_model.get_acq_values(X_test).data

            data = [(x.item(), y.item(), z.item()) for x,y,z in zip(X_test.T[0],
                                                                    X_test.T[1],
                                                                    acq_values)]

            acq_plot = hv.HeatMap(data).opts(title='Acquisition function',
                                             xlabel='continous variable',
                                             ylabel='continous context',
                                             cmap='plasma',
                                             colorbar=True,
                                             width=580,
                                             show_legend=True,
                                            ) * train_points * context_marker * next_point
            acq_plot = acq_plot.opts(legend_position='left')

            unc_pred = init_model(X_test)[1].data

            data = [(x.item(), y.item(), z.item()) for x,y,z in zip(X_test.T[0],
                                                                    X_test.T[1],
                                                                    unc_pred)]

            unc_plot = hv.HeatMap(data).opts(title='Uncertainty of prediction',
                                             xlabel='continuous variable',
                                             ylabel='continuous context',
                                             cmap='RdBu_r',
                                             colorbar=True,
                                             width=380) * train_points * context_marker * next_point
            unc_plot = unc_plot.opts(show_legend=False)

            model_dict[i] = ((mean_plot*next_point).opts(show_legend=False, fontsize=fontsize) +
                             unc_plot.opts(fontsize=fontsize) +
                             acq_plot.opts(fontsize=fontsize)).opts(shared_axes=False).cols(2)

    return model_dict

In [ ]:
lengthscale = []
outputscale = []
noise_ = []

hmap3 = hv.HoloMap(getHeatMap(bo_model, X_test, get_current_context, Objective2D, num_iter,
                              init_samples=(X_init, y_init)), kdims='iteration')
hv.output(hmap3.collate(), widget_location='top')

In [ ]:
hv.extension('bokeh')

hv.save(hmap3.collate(), 'contextExtended.auto')

In [ ]:
lengthscale_ex_plot = hv.Curve(((torch.tensor(lengthscale))),
                               label='extended BO').opts(xlabel='iteration', ylabel=' ', height=200)
outputscale_ex_plot = hv.Curve((torch.tensor(outputscale)),
                               label='extended BO').opts(xlabel='iteration', ylabel=' ', height=200)
noise_ex_plot = hv.Curve((torch.tensor(noise_)),
                         label='extended BO').opts(xlabel='iteration', ylabel=' ', height=200)

In [ ]:
hv.extension('plotly')

num_init = 1

acq_curve = hv.Curve((x1, acqs[num_init][0])).opts(xlim=(-0.1, 5.1),
                                                   title='Context value: {:.2f}'.format(acqs[num_init][1]))
acq_next = torch.cat((x1[int(acqs[num_init][0].argmax())].reshape(-1), acqs[num_init][0].max().reshape(-1)))
(acq_curve * hv.Scatter((acq_next[0], acq_next[1])).opts(color='red', size=9)).opts(height=300, width=600)

## Extended BO - noisy

In [ ]:
noise=0.5

y_init = Objective2D(x1=X_init.T[0:1].T, x2=X_init.T[1:].T, noise=noise)
kernel = gp.kernels.RBF(input_dim=input_dim)

bo_model = PyroBO(X_init, y_init, kernel=kernel, acq_fun=ConfidenceBound(input_dim=input_dim,
                                                                         kappa=kappa,
                                                                         maximize=True))
bo_model.update()

In [ ]:
hv.extension('bokeh')

lengthscale = []
outputscale = []
noise_ = []

hmap4 = hv.HoloMap(getHeatMap(bo_model, X_test, get_current_context, Objective2D, num_iter,
                              init_samples=(X_init, y_init)), kdims='iteration')
hv.output(hmap4.collate(), widget_location='top')

In [ ]:
hv.save(hmap4.collate(), 'contextExtended_noisy.auto')

In [ ]:
lengthscale_exn_plot = hv.Curve(((torch.tensor(lengthscale))),
                                label='extended BO').opts(xlabel='iteration', ylabel=' ', height=200)
outputscale_exn_plot = hv.Curve((torch.tensor(outputscale)),
                                label='extended BO').opts(xlabel='iteration', ylabel=' ', height=200)
noise_exn_plot = hv.Curve((torch.tensor(noise_)),
                          label='extended BO').opts(xlabel='iteration', ylabel=' ', height=200)

In [ ]:
hv.extension('plotly')

vline = hv.VLine(4).opts(line_color='red')
vline_cl = hv.VLine(14).opts(line_color='red', line_dash='dot')
vline_noisy = hv.VLine(9).opts(line_color='red')
vline_cl_noisy = hv.VLine(6).opts(line_color='red', line_dash='dot')

(lengthscale_ex_plot.opts(height=200, color='blue', dash='solid', show_legend=True) * 
 lengthscale_cl_plot.opts(color='blue', dash='dot', show_legend=True, title='length scale of noise free data') +
 lengthscale_cln_plot.opts(color='blue', dash='dot', title='length scale of noisy data') * 
 lengthscale_exn_plot.opts(color='blue', dash='solid')).cols(1)

In [ ]:
(outputscale_ex_plot.opts(height=200, color='blue', dash='solid', show_legend=True) * 
 outputscale_cl_plot.opts(color='blue', dash='dot', show_legend=True, title='output scale of noise free data') +
 outputscale_cln_plot.opts(color='blue', dash='dot', title='output scale of noisy data') * 
 outputscale_exn_plot.opts(color='blue', dash='solid')).cols(1)

In [ ]:
hline = hv.HLine(0.5).opts(line_color='black')

(noise_ex_plot.opts(height=200, color='blue', dash='solid', show_legend=True) * 
 noise_cl_plot.opts(color='blue', dash='dot', show_legend=True, title='noise of noise free data') +
 (noise_cln_plot.opts(color='blue', dash='dot', title='noise of noisy data') *
  noise_exn_plot.opts(color='blue', dash='solid') * 
  hline)).cols(1)